In [1]:
%%capture
from sensitive_data.sensitive_data import *
from faker import Faker
import pandas as pd

# Create fake data 

In [2]:
fake = Faker()
df = pd.DataFrame([fake.profile() for i in range(100)])
df.head()

,address,birthdate,blood_group,company,current_location,job,mail,name,residence,sex,ssn,username,website
0,"27296 Melissa Forges Suite 225\nEast Keith, GA...",1907-07-19,A+,Hayes-Larson,"(20.652527, -19.086214)",Chief Operating Officer,maylaura@gmail.com,John Walsh,"46825 Steele Turnpike\nMcconnellshire, NC 09514",M,440-14-6811,iflores,"[http://hernandez.biz/, http://www.oconnor-gil..."
1,"170 Medina Trail Suite 330\nLake Sarahview, GA...",2006-08-22,O-,Miller and Sons,"(50.3729815, 12.196201)",Educational psychologist,dlambert@yahoo.com,Dr. Andrew Campbell,"305 Ward Pine Apt. 653\nWilsonfurt, AZ 81350",M,869-24-0835,lori38,[https://campbell.org/]
2,"3413 Garcia Key\nPort Lindsayview, IL 23687",2014-10-15,AB+,"Harding, Park and Wheeler","(30.840559, -89.108446)",Bookseller,matthew08@gmail.com,Daniel Lewis,Unit 2272 Box 4331\nDPO AE 33958,M,110-16-2282,brittany64,[http://www.davis-lawson.net/]
3,"49208 James Unions Apt. 552\nKatelynmouth, KY ...",1956-05-10,B+,Vazquez and Sons,"(38.0015855, 157.282533)",Control and instrumentation engineer,seanrussell@yahoo.com,Suzanne Myers DVM,"19276 Jennifer Centers\nWest Andrew, FL 08560",F,731-90-6964,cathy90,"[https://soto.com/, https://www.stein.info/, h..."
4,"40028 Shawn Crossroad\nNew Alexandria, SC 40318",2008-06-17,AB-,Jenkins-Wade,"(-48.032880, -91.999161)","Librarian, public",qbradshaw@hotmail.com,Thomas Cole,3655 Murphy Extensions Suite 508\nEast Breanna...,M,694-25-8882,boonerhonda,"[https://www.osborn.com/, http://bryant.com/]"


# Redact sensitive data

In [3]:
sensitive_data(df, redact=True, score_threshold=0.50).head(5)

,address,birthdate,blood_group,company,current_location,job,mail,name,residence,sex,ssn,username,website
0,"27296 <PERSON> Forges Suite 225\n<LOCATION>, <...",1907-07-19,A+,<PERSON>,"(Decimal('20.652527'), Decimal('-19.086214'))",Chief Operating Officer,<EMAIL_ADDRESS>,<PERSON>,"46825 Steele Turnpike\nMcconnellshire, <LOCATI...",M,<US_SSN>,iflores,"['http://<DOMAIN_NAME>/', 'http://<DOMAIN_NAME..."
1,"170 <LOCATION> Suite 330\nLake Sarahview, GA 4...",<DATE_TIME>,O-,Miller and Sons,"(Decimal('50.3729815'), Decimal('12.196201'))",Educational psychologist,<EMAIL_ADDRESS>,Dr. <PERSON>,"305 Ward Pine Apt. 653\nWilsonfurt, <LOCATION>...",M,<US_SSN>,lori38,['https://<DOMAIN_NAME>/']
2,"3413 <PERSON> Key\nPort Lindsayview, IL 23687",<DATE_TIME>,AB+,"<LOCATION>, Park and <LOCATION>","(Decimal('30.840559'), Decimal('-89.108446'))",Bookseller,<EMAIL_ADDRESS>,<PERSON>,Unit 2272 Box 4331\nDPO AE 33958,M,<US_SSN>,brittany64,['http://<DOMAIN_NAME>/']
3,"49208 James Unions Apt. 552\nKatelynmouth, <LO...",<DATE_TIME>,B+,Vazquez and Sons,"(Decimal('38.0015855'), Decimal('157.282533'))",Control and instrumentation engineer,<EMAIL_ADDRESS>,<PERSON>,"19276 <PERSON> Centers\n<LOCATION>, FL 08560",F,<US_SSN>,cathy90,"['https://<DOMAIN_NAME>/', 'https://<DOMAIN_NA..."
4,"40028 <PERSON>\n<LOCATION>, SC 40318",<DATE_TIME>,AB-,<PERSON>,"(Decimal('-48.032880'), Decimal('-91.999161'))","Librarian, public",<EMAIL_ADDRESS>,<PERSON>,"3655 Murphy Extensions Suite 508\n<PERSON>, DC...",M,<US_SSN>,boonerhonda,"['https://<DOMAIN_NAME>/', 'http://<DOMAIN_NAM..."


# Redact sensitive data using selected columns

In [4]:
cols = ['birthdate', 'mail']
df[cols] = sensitive_data(df, redact=True, score_threshold=0.50, cols=['birthdate', 'mail'])
df.head(2)

,address,birthdate,blood_group,company,current_location,job,mail,name,residence,sex,ssn,username,website
0,"27296 Melissa Forges Suite 225\nEast Keith, GA...",1907-07-19,A+,Hayes-Larson,"(20.652527, -19.086214)",Chief Operating Officer,<EMAIL_ADDRESS>,John Walsh,"46825 Steele Turnpike\nMcconnellshire, NC 09514",M,440-14-6811,iflores,"[http://hernandez.biz/, http://www.oconnor-gil..."
1,"170 Medina Trail Suite 330\nLake Sarahview, GA...",<DATE_TIME>,O-,Miller and Sons,"(50.3729815, 12.196201)",Educational psychologist,<EMAIL_ADDRESS>,Dr. Andrew Campbell,"305 Ward Pine Apt. 653\nWilsonfurt, AZ 81350",M,869-24-0835,lori38,[https://campbell.org/]


In [5]:
df['birthdate'].value_counts()

<DATE_TIME>    92
1970-11-04      1
1913-07-13      1
1936-11-15      1
2015-09-20      1
1907-07-19      1
2016-12-14      1
1911-04-14      1
1961-11-25      1
Name: birthdate, dtype: int64

In [6]:
df['mail'].value_counts()

<EMAIL_ADDRESS>    100
Name: mail, dtype: int64

# Identify infotypes in each column

In [7]:
fake = Faker()
df = pd.DataFrame([fake.profile() for i in range(100)])
df.head()

,address,birthdate,blood_group,company,current_location,job,mail,name,residence,sex,ssn,username,website
0,"58197 Riley Corner\nLake Amy, WA 58762",1981-06-25,AB+,Hernandez PLC,"(-17.587400, 122.149265)",Product/process development scientist,randyhall@gmail.com,Connie Smith,"70791 Johnson Ridge Suite 018\nJacksonmouth, A...",F,615-07-0274,gravesmelissa,"[http://www.jones-adams.com/, https://www.cart..."
1,"34134 Wilson Mountain\nBradleyberg, ME 26750",1958-09-01,O+,"Tyler, Jennings and Kennedy","(83.624481, -13.433166)",Automotive engineer,ugomez@yahoo.com,Kristen Parker,"764 Hart Locks Apt. 627\nCisnerosberg, GA 31216",F,790-81-2056,trevor32,"[http://miller.org/, http://www.watson-west.in..."
2,"642 Johnson Divide\nDavidland, SC 53405",1932-12-19,A-,"Johns, Boyer and Mendoza","(-85.7457695, 17.087065)","Research officer, trade union",tracysmith@gmail.com,Ross Wells,"0964 Abigail Shores\nNorth Petermouth, DE 77829",M,431-84-8599,lynn54,"[https://www.raymond.com/, https://www.flynn.i..."
3,"576 Dawn Hollow Apt. 368\nPort Steven, WV 69267",1964-08-08,AB+,Anderson LLC,"(85.2788825, -150.179609)",Clothing/textile technologist,keith78@yahoo.com,Christina Ross,"2454 Parsons Lodge Apt. 509\nSouth Nathanview,...",F,283-09-3406,lauren08,[https://www.salas.com/]
4,"751 Eric Canyon\nNorth Jeffrey, IA 53805",2000-02-24,O+,"Parker, Dawson and Franklin","(-47.803523, 148.056151)",Theatre stage manager,charleswatson@hotmail.com,Barry Harper,"3167 Burgess Stream Apt. 231\nNew Douglas, DE ...",M,343-38-6895,clarkbethany,"[http://www.smith-evans.org/, http://hartman.o..."


In [8]:
sensitive_data(df, redact=False, identify_column_infotypes=True, score_threshold=0.0, sample_size=100)

{'address': ['DATE_TIME', 'LOCATION', 'PERSON', 'US_DRIVER_LICENSE'],
 'birthdate': ['DATE_TIME'],
 'blood_group': [],
 'company': ['LOCATION', 'PERSON'],
 'current_location': ['LOCATION', 'NRP', 'PERSON'],
 'job': ['DATE_TIME', 'PERSON'],
 'mail': ['DOMAIN_NAME', 'EMAIL_ADDRESS'],
 'name': ['LOCATION', 'PERSON'],
 'residence': ['DATE_TIME', 'LOCATION', 'NRP', 'PERSON', 'US_DRIVER_LICENSE'],
 'sex': [],
 'ssn': ['US_SSN'],
 'username': ['LOCATION', 'PERSON'],
 'website': ['DATE_TIME', 'DOMAIN_NAME', 'LOCATION', 'PERSON']}